In [212]:
#!dir

In [222]:
#from ..private import config_spotipy
import sys
from configspy import *
import spotipy
import json
import pandas as pd
import time

from spotipy.oauth2 import SpotifyClientCredentials


In [223]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [224]:
def extract_song_id(uri):
    # Split the URI by ':'
    parts = uri.split(':')
    # The song ID is the last part
    song_id = parts[-1]
    return song_id


In [225]:
def get_audio_features(list_of_song_ids):
    audio_features = []
    chunk_size = 50  # Define the chunk size

    try:
        for i in range(0, len(list_of_song_ids), chunk_size):
            chunk = list_of_song_ids[i:i+chunk_size]
            # Get audio features for the chunk of song_ids
            features = sp.audio_features(tracks=chunk)
            audio_features.extend(features)
            if i + chunk_size < len(list_of_song_ids):
                time.sleep(20)  # Add a 20-second timeout between chunks
    except Exception as e:
        print(f"Error: {e}")

    return audio_features


In [226]:
def extract_audio_feature_keys(features):
    # Extract the keys (audio feature names) from the first item in the list
    return list(features[0].keys())


In [237]:
# Function to search for a song and return its URIs (Spotify IDs)
def search_song(title, artist, limit=1):
    song_uris = []  # Initialize a list to store song URIs

    try:
        if artist:
            # Search for the song using both title and artist
            query = f"track:{title} artist:{artist}"
        else:
            # Search for the song using title alone
            query = f"track:{title}"

        # Add a delay before making the API call
        # Perform the search
        results = sp.search(q=query, type='track', limit=limit)

        # Check if any tracks were found
        if results['tracks']['items']:
            # Create a set to keep track of unique artist names
            unique_artists = set()
            
            for track in results['tracks']['items']:
                uri = track['uri']
                artist_name = track['artists'][0]['name']

                if artist_name not in unique_artists:
                    # Append the URI if it's the first instance of the artist
                    song_uris.append(uri)
                    unique_artists.add(artist_name)

        # Create a dictionary to store song data
        song_data = {
            'song_name': [],
            'artist': [],
        }

        for uri in song_uris:
            # Find the corresponding track in the search results
            track = next(item for item in results['tracks']['items'] if item['uri'] == uri)
            song_data['song_name'].append(track['name'])
            song_data['artist'].append(track['artists'][0]['name'])
            #song_data['id'].append(extract_song_id(track['uri']))

        # Create a DataFrame from the song data
        song_df = pd.DataFrame(song_data)

    except Exception as e:
        # Handle exceptions, e.g., if there's an issue with the Spotify API
        print(f"Error: {e}")
        song_df = pd.DataFrame()

    return song_uris, song_df


In [238]:
# Function to get song info with audio features
def get_song_info_with_features(title, artist, limit):
    # Search for song URIs and song_df
    song_uris, song_df = search_song(title, artist, limit)

    if not song_uris:
        return pd.DataFrame()  # Return an empty DataFrame if no song URIs are found

    # Retrieve audio features for all song URIs
    audio_features = get_audio_features(song_uris)

    # Convert the audio features into a DataFrame
    audio_features_df = pd.DataFrame(audio_features)

    # Concatenate the two DataFrames
    song_info_df = pd.concat([song_df, audio_features_df], axis=1)

    return song_info_df


In [239]:
#add to the existing dataset
def add_audio_features(df, audio_features_df):
    # Check if both dataframes have the same length
    if len(df) != len(audio_features_df):
        raise ValueError("Dataframes must have the same length.")

    # Concatenate the dataframes horizontally
    extended_df = pd.concat([df, audio_features_df], axis=1)
    extended_df.

    return extended_df


In [240]:
def add_audio_features(audio_features_df, not_top100_df):
    # Merge the two dataframes on the 'uri' column
    extended_df = pd.merge(audio_features_df, not_top100_df, on='uri', how='outer')
    extended_df.to_csv(r'..\data\not_top100.csv', index=False) # update not_top100 csv
    return extended_df

In [248]:
#load previous sets:
def songlists_loader(): #load ->top_100, not_top_100 dataFrames from csv in ..\data\
    top_100 = pd.read_csv(r'..\data\top100.csv')
    not_top_100 = pd.read_csv(r'..\data\not_top100.csv')
    return top_100, not_top_100

In [242]:
title = "Georgia off My Mind"
artist = ""
limit = 5

In [243]:
tracks=get_song_info_with_features(title, artist, limit)
tracks

,song_name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Georgia Off My Mind,Larkin Poe,0.643,0.599,2,-8.541,1,0.0292,0.0214,0.000002,0.2820,0.591,113.982,audio_features,4dLPPSxHgaoy38YmX4yJnV,spotify:track:4dLPPSxHgaoy38YmX4yJnV,https://api.spotify.com/v1/tracks/4dLPPSxHgaoy...,https://api.spotify.com/v1/audio-analysis/4dLP...,234212,4
1,Georgia off My Mind,Jared Wade,0.587,0.526,0,-6.738,1,0.0245,0.4620,0.000515,0.1240,0.282,143.849,audio_features,61QrMlxsE6A0Qyw21RGmbZ,spotify:track:61QrMlxsE6A0Qyw21RGmbZ,https://api.spotify.com/v1/tracks/61QrMlxsE6A0...,https://api.spotify.com/v1/audio-analysis/61Qr...,234587,4
2,Georgia on My Mind (with Gene Krupa & His Orch...,Anita O'Day,0.571,0.302,7,-8.534,0,0.0341,0.9200,0.000173,0.0979,0.603,137.139,audio_features,4dmyrJKIeq1Y2XYS48SDSA,spotify:track:4dmyrJKIeq1Y2XYS48SDSA,https://api.spotify.com/v1/tracks/4dmyrJKIeq1Y...,https://api.spotify.com/v1/audio-analysis/4dmy...,174200,4


In [249]:
top_100, not_top_100 = songlists_loader()
display(top_100.head(),not_top_100.head())

,song_name,artist,hot
0,Paint The Town Red,Doja Cat,H
1,Snooze,SZA,H
2,Cruel Summer,Taylor Swift,H
3,Fast Car,Luke Combs,H
4,3D,Jung Kook & Jack Harlow,H


,song_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,hot
0,Mercury: Retrograde,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,...,156.985,audio_features,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,N
1,Pathology,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,...,115.080,audio_features,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,N
2,Symbiote,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,...,218.050,audio_features,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,N
3,ProductOfDrugs (Prod. The Virus and Antidote),0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,...,186.948,audio_features,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,N
4,Venom,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,...,147.988,audio_features,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,N


In [244]:
updated_song_list=add_audio_features(not_top_100,tracks)
updated_song_list

,song_name_x,danceability_x,energy_x,key_x,loudness_x,mode_x,speechiness_x,acousticness_x,instrumentalness_x,liveness_x,...,instrumentalness_y,liveness_y,valence_y,tempo_y,type_y,id_y,track_href_y,analysis_url_y,duration_ms_y,time_signature_y
0,Mercury: Retrograde,0.831,0.814,2.0,-7.364,1.0,0.420,0.0598,0.013400,0.0556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mercury: Retrograde,0.831,0.814,2.0,-7.364,1.0,0.420,0.0598,0.013400,0.0556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mercury: Retrograde,0.831,0.814,2.0,-7.364,1.0,0.420,0.0598,0.013400,0.0556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mercury: Retrograde,0.831,0.814,2.0,-7.364,1.0,0.420,0.0598,0.013400,0.0556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mercury: Retrograde,0.831,0.814,2.0,-7.364,1.0,0.420,0.0598,0.013400,0.0556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,Born 2 Live,0.785,0.648,7.0,-6.907,1.0,0.279,0.0157,0.001980,0.1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21490,Can't Hold On,0.747,0.832,3.0,-5.737,0.0,0.185,0.2330,0.000014,0.1300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000002,0.2820,0.591,113.982,audio_features,4dLPPSxHgaoy38YmX4yJnV,https://api.spotify.com/v1/tracks/4dLPPSxHgaoy...,https://api.spotify.com/v1/audio-analysis/4dLP...,234212.0,4.0
21492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000515,0.1240,0.282,143.849,audio_features,61QrMlxsE6A0Qyw21RGmbZ,https://api.spotify.com/v1/tracks/61QrMlxsE6A0...,https://api.spotify.com/v1/audio-analysis/61Qr...,234587.0,4.0
